In [30]:
# 모듈, 경로 설정
import pandas as pd
import pydeck as pdk
import geopandas as gpda
import config
import json
import requests
import sys
import folium

path = "C:\github/aide-analysis/"
sys.path.append(path)

path2 = "C:\github/prop-tech/"
sys.path.append(path2)
from src.sql import DatabaseHandler
db = DatabaseHandler()

from core.config import Config
config = Config()


In [ ]:
# Vworld 시군구 경계 API 호출
def gl(sig_cd):
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 행정동 경계
    data = "LT_C_ADSIGG_INFO"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f"sig_cd:like:{11}"
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"

    # 요청 결과
    res = json.loads(requests.get(url).text)

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current == page_total:
            break
    featureCollection["features"] = features
    geo = featureCollection
    return geo
geo = gl(11)

In [ ]:
# Vworld 시군구 경계 API 호출
def gl2(sig_cd):
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 행정동 경계
    data = "LT_C_ADSIGG_INFO"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f"sig_cd:=:{11650}"
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"

    # 요청 결과
    res = json.loads(requests.get(url).text)

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current == page_total:
            break
    featureCollection["features"] = features
    geo = featureCollection
    return geo
geo2 = gl2(11)

In [ ]:
# params
center = [37.541, 126.986]
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    width = 750,
    height = 500,
    tiles = tiles[2]
)


# Marker
"""
location : 마커를 추가할 경위도 좌표/ 
popup : 표기할 팝업문구(클릭시 문구)/ 
tooltip : 표기할 툴팁 지정(마우스 오버))
"""
folium.Marker(
    location = [lat, lon],
    popup = '반포자이',
    tooltip = '반포자이',
    icon = folium.Icon('red', icon = icon[0])
).add_to(m)

# GeoJson 활용
folium.GeoJson(
    geo,
    name = '서초구',
    style_function = lambda x: {'fillColor':'#00000000' ,'color':'black','weight' :'1'}
).add_to(m)


folium.GeoJson(
    geo2,
    name = '서초구',
    style_function = lambda x: {'fillColor':'red' ,'color':'black','weight' :'1','fillOpacity':'0.3'}
).add_to(m)

m
# m.save('data/result.html')

### DB연결후 적용

#### 세대수

In [ ]:
# 데이터 불러오기
sql = f"""
SELECT hdongName, 
       totalSaedae 
FROM aide.population_saedae
WHERE sidoCode ='11' 
      and sigunguCode != '11000' 
      and date = (SELECT MAX(date) 
                  FROM aide.population_saedae);
"""
df = db.read_table(sql)

In [ ]:

# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[2]
)

# Marker
folium.Marker(
    location = [37.5146321, 127.018293],
    popup = '회사',
    tooltip = '회사',
    icon = folium.Icon('red', icon = icon[0])
).add_to(m)


# Marker
folium.Marker(
    location = [37.4857983, 126.948162],
    popup = '집',
    tooltip = '집',
    icon = folium.Icon('red', icon = icon[0])
).add_to(m)


# Polygon
m.choropleth(
    geo_data = geo,
    data = df,
    columns = ('hdongName','totalSaedae'),
    key_on = 'feature.properties.full_nm',
    fill_color = 'YlOrRd',
    fill_opacity = 0.5,
    line_opacity = 0.2
)
m

In [ ]:
location_data = [[37.5146321, 127.018293],
                [37.4857983, 126.948162]]
folium.PolyLine(locations = location_data, tooltip = "Polyline").add_to(m)
m

### 연속지적도 + 아파트 전세 공급평단가(최근날짜기준)

In [ ]:
# 최근실거래가 기준 아파트 공급평단가

sigunguCode = 41463
sql = f"""
SELECT T1.pnuCode,
	   T1.contractYear,
       T1.contractMonth,
       T1.tradingSupplyUnitPriceAverage,
       T1.jeonseSupplyUnitPriceAverage,
       T1.wolseSupplyUnitPriceAverage,
       T1.rn
FROM(
    -- 단지별 년,월별 평균거래가의 평균
    SELECT pnuCode, 
           contractYear, 
           contractMonth,
           AVG(tradingSupplyUnitPriceAverage) as tradingSupplyUnitPriceAverage,
           AVG(jeonseSupplyUnitPriceAverage) AS jeonseSupplyUnitPriceAverage,
           AVG(wolseSupplyUnitPriceAverage) AS wolseSupplyUnitPriceAverage,
           row_number() OVER(PARTITION BY pnuCode ORDER BY contractYear DESC) AS rn

    FROM aide.apartment_danji_monthly_price
    WHERE pnuCode LIKE '{sigunguCode}%'
    GROUP BY pnuCode, contractYear, contractMonth 

    ) AS T1
WHERE T1.rn=  1
"""
price_df = db.read_table(sql)

In [ ]:
# 전세가 공급평단가를 기준으로 측정하기 위해 결측값제거
price_df = price_df.dropna(subset = 'jeonseSupplyUnitPriceAverage')
price_df = price_df[['pnuCode','contractYear','contractMonth','jeonseSupplyUnitPriceAverage']]
price_df

In [ ]:
# API를 호출할 수 있는 형태로 변형
lst = ",".join(price_df['pnuCode'])

In [ ]:
def continue_land():
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 연속지적도
    data = "LP_PA_CBND_BUBUN"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f'pnu:IN:{lst}'
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"

    # 요청 결과
    res = json.loads(requests.get(url).text)

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current== page_total:
            break 
    featureCollection["features"] = features
    geo = featureCollection
    return geo

geo = continue_land()

In [ ]:

# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[0]
)


# Polygon
m.choropleth(
    geo_data = geo,
    data = price_df,
    columns = ('pnuCode','jeonseSupplyUnitPriceAverage'),
    key_on = 'feature.properties.pnu',
    fill_color = 'YlOrRd',
    fill_opacity = 0.5,
    line_opacity = 0.2
)
m

시군구별 전세 전용 평단가 평균

In [ ]:
sql33 = f"""
SELECT T1.sigunguCode, T1.contractYear, T1.contractMonth, T1.tradingJeonyongUnitPriceAverage, T1.jeonseJeonyongUnitPriceAverage, T1.wolseJeonyongUnitPriceAverage, T1.rn
FROM(
SELECT sigunguCode, 
		contractYear, 
		contractMonth, 
		AVG(tradingJeonyongUnitPriceAverage) AS tradingJeonyongUnitPriceAverage , 
		AVG(jeonseJeonyongUnitPriceAverage) AS jeonseJeonyongUnitPriceAverage, 
		AVG(wolseJeonyongUnitPriceAverage) AS wolseJeonyongUnitPriceAverage,
		row_number() OVER(PARTITION BY sigunguCode ORDER BY contractYear DESC) AS rn
FROM aide.apartment_danji_monthly_price
GROUP BY sigunguCode, contractYear, contractMonth
) AS T1
WHERE T1.rn = 1
"""
test = db.read_table(sql33)

In [ ]:
test[test['sigunguCode']=="42810"]

In [ ]:
# join
lst = ",".join(test['sigunguCode'])

In [ ]:
# Vworld 시군구 경계 API 호출
def gl(sig_cd):
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 행정동 경계
    data = "LT_C_ADSIGG_INFO"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f"sig_cd:IN:{lst}"
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"

    # 요청 결과
    res = json.loads(requests.get(url).text)

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current == page_total:
            break
    featureCollection["features"] = features
    geo = featureCollection
    return geo
geo = gl(11)

In [ ]:

# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[0]
)


# Polygon
m.choropleth(
    geo_data = geo,
    data = test,
    columns = ('sigunguCode','jeonseJeonyongUnitPriceAverage'),
    key_on = 'feature.properties.sig_cd',
    fill_color = 'YlOrRd',
    fill_opacity = 0.5,
    line_opacity = 0.2
)
m

- 최근에 거래가 전세거래가 아닌 거래(매매 혹은 월세)가 있던 지역은 회색
- 현재는 가장 최근 거래에 따라서 조회를 하기 때문에 매매, 전세, 혹은 월세 중 없는 거래가 있을 수 있으므로 특정 거래방식의 선택에따라 없는 부분이 생기지만 향후 필요한 거래형태에 따라 특정기간내 특정형태거래를 조회하는 쿼리를 만들면 모든 시군구를 채울 수 있음 

### 시군구경계 + 미분양 시각화

#### sig_cd 로 API를 호출하게 되면 시군구 뎁스가 맞지않아 모든 시군구가 매핑이 되지않고, full_nm으로 API를 호출하면 "JSONDecodeError: Expecting value: line 1 column 1 (char 0)" 에러가 발생한다. 일시 중단

In [ ]:
sql = f"""

SELECT * 
FROM aide.unsold_sigungu
WHERE year = '2022' 
      AND month = '03'
      AND sigunguName != '계'
      
"""
df = db.read_table(sql)
df

In [ ]:
lst = ','.join(df['sigunguCode'])

In [ ]:
# Vworld 시군구 경계 API 호출
import xmltodict
def gl():
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 행정동 경계
    data = "LT_C_ADSIGG_INFO"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f"sig_cd:LIKE:{11}%"
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"


    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        
        # res = xmltodict.parse(requests.get(url).text)
        # res = json.loads(json.dumps(res))
        
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current == page_total:
            break
    featureCollection["features"] = features
    geo = featureCollection
    return geo
geo = gl()

In [ ]:
# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[0]
)


# Polygon
m.choropleth(
    geo_data = geo,
    data = df,
    columns = ('sigunguCode','unsold'),
    key_on = 'feature.properties.sig_cd',
    fill_color = 'YlGnBu',
    fill_opacity = 0.5,
    line_opacity = 0.2
)
m

### 입력한 지번 반경에 있는 비슷한 스펙의 매물을 조회 및 건물 지하철등 상업시설조회 

**구현하고자 하는 기능:**
1. ~~지번을 입력받으면 해당 지번 물건의 반경 안에 있는 매물들을 모두 지도상에 표시한다~~
2. ~~매물들의 세부정보 popup에 추가~~
3. 해당 지번은 과 비슷한 스펙의 주변 매물들이 지도상에 표시가 된다.
4. ~~마우스를 올리면 매물에 대한 정보가 요약된다. ~~
5. ~~주변 지하철이 표시가 된다.~~
6. 주변 공원 및 큰 상점들이 표시가 된다.

In [5]:
# 경로설정
import sys
path = "C:\github/aide-analysis/"
sys.path.append(path)

# 모듈 import
import pandas as pd
import src
import folium
from src.openapi import KakaoLocalAPI as kakao
from openpyxl import load_workbook
from src.utils import insert_dataframe_to_excel_sheet
from src.sql import DatabaseHandler
from src.temp import *


db = DatabaseHandler()

# 카카오 API 호출
docs = kakao.search_address('서울시 서초구 반포동 20-43')


# 파싱정보
# 지번 추출
jibeon = docs['documents'][0]['address_name']

# 시도/ 시군구/ 행정동 코드 추출
sidoCode = docs["documents"][0]["address"]["h_code"][:2]
sigunguCode = docs["documents"][0]["address"]["h_code"][:5]
hdongCode = docs["documents"][0]["address"]["h_code"]

# 행정구역 명칭
sido_ = docs["documents"][0]["address"]["region_1depth_name"]
sigungu_ = docs["documents"][0]["address"]["region_2depth_name"]
if sigungu_ == "":
    sigungu_ = sido_
hdong_ = docs['documents'][0]['address']['region_3depth_h_name']
if hdong_ == "":
    hdong_ = docs['documents'][0]['address']['region_3depth_name']

# 경도, 위도
lon = docs['documents'][0]['address']['x'] 
lat = docs['documents'][0]['address']['y'] 
radius = 1000

# 단지명
danjiName = docs['documents'][0]['road_address']['building_name']

# 시계열 인자
startyear = datetime.now().year-10
endyear = datetime.now().year-1



In [23]:
# folium용
# 입력된지번의 일정 반경내 아파트와 아파트 정보
sql = f"""
WITH tmp AS (  SELECT A.pnuCode, 
                      addressJibeon, 
                      addressRoad, 
                      danjiName, 
                      usePermissionDate, 
                      B.contractYear, 
                      B.contractMonth, 
                      B.jeonseJeonyongUnitPriceAverage, 
                      buildingAreaPerLandArea, 
                      floorAreaRatio, 
                      mainPurposeCodeName, 
                      otherPurpose, 
                      saedaeCount, 
                      totalParkingLotCount, 
                      lon, 
                      lat, 
                      coord, 
                      geom
               FROM aide.apartment_danji_information AS A
               JOIN (SELECT T1.*
                     FROM(
                         SELECT pnuCode,
                                contractYear,
                                contractMonth,
                                supplyArea,
                                tradingJeonyongUnitPriceAverage,
                                jeonseJeonyongUnitPriceAverage,
                                wolseJeonyongUnitPriceAverage,
                                ROW_NUMBER() OVER(PARTITION BY pnuCode  ORDER BY contractYear DESC, contractMonth DESC) AS num
                         FROM aide.apartment_danji_monthly_area_price
                         WHERE pnuCode LIKE '{sidoCode}%' AND jeonseJeonyongUnitPriceAverage is not null
                        ) AS T1
                     WHERE T1.num = 1 
                   ) AS B
                   on A.pnuCode = B.pnuCode 
                   
)SELECT tmp.*,
        ST_Distance_Sphere(POINT({lon},{lat}),geom) AS distance
FROM tmp
WHERE ST_Distance_Sphere(POINT({lon},{lat}), geom) <={radius}
ORDER BY distance ASC;
"""
df= db.read_table(sql)


# 반경내 지하철역 정보 테이블
sql2 = f"""
SELECT * , 
       ST_Distance_Sphere(POINT({lon},{lat}),POINT(lon,lat)) AS distance
FROM aide.railway_station_info
WHERE ST_Distance_Sphere(POINT({lon},{lat}),POINT(lon,lat)) <= {radius}
ORDER BY distance ASC;
"""
train = db.read_table(sql2)

In [26]:
df

,pnuCode,addressJibeon,addressRoad,danjiName,usePermissionDate,contractYear,contractMonth,jeonseJeonyongUnitPriceAverage,buildingAreaPerLandArea,floorAreaRatio,mainPurposeCodeName,otherPurpose,saedaeCount,totalParkingLotCount,lon,lat,coord,geom,distance
0,1165010600100670002,서울특별시 서초구 잠원동 67-2번지,서울특별시 서초구 잠원로 46-38,브라운스톤잠원,20030925,2021,12,3677.851356,25,297,공동주택,공동주택(아파트),83,94,127.009163,37.508170,POINT(127.009162726394 37.5081699251029),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x15\x8d...,261.025417
1,1165010600100590003,서울특별시 서초구 잠원동 59-3번지,서울특별시 서초구 신반포로 261,잠원현대훼밀리,19970827,2022,3,3607.723859,25,342,공동주택,공동주택,113,113,127.012445,37.508982,POINT(127.012444652076 37.5089823037232),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00Lk\xa7\x...,317.841384
2,1165010600100670001,서울특별시 서초구 잠원동 67-1번지,서울특별시 서초구 잠원로 46-4,데뜨아르,20080422,2022,4,4281.096817,24,211,공동주택,공동주택,22,26,127.008663,37.508635,POINT(127.008663293929 37.5086345196832),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00r\x06}\x...,328.906576
3,1165010600101570000,서울특별시 서초구 잠원동 157번지,서울특별시 서초구 신반포로33길 15,동아,,2022,4,3893.281267,23,316,공동주택,공동주택(아파트),991,1025,127.010929,37.509316,POINT(127.010929282174 37.5093163798427),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00d`\xbb\x...,329.775004
4,1165010700100200043,서울특별시 서초구 반포동 20-43번지,서울특별시 서초구 신반포로 270,반포자이,20090313,2022,4,4596.193480,14,270,공동주택,공동주택(아파트),3410,6075,127.013900,37.508401,POINT(127.013900193294 37.5084007014215),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\xc6\xe6...,340.285957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,1165010600100760002,서울특별시 서초구 잠원동 76-2번지,서울특별시 서초구 반포대로 310,두산위브,20030625,2021,12,3548.702242,36,299,공동주택,공동주택(아파트),49,49,127.000597,37.506646,POINT(127.000597344152 37.5066461134614),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00:fq\xc9\...,921.244489
71,1165010600101590000,서울특별시 서초구 잠원동 159번지,서울특별시 서초구 잠원로 117,아크로리버뷰신반포,20180604,2022,4,6737.905985,16,276,공동주택,공동주택(아파트),595,890,127.007283,37.514498,POINT(127.007283232196 37.5144979558146),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\xd3\x05...,964.352625
72,1165010600100630000,서울특별시 서초구 잠원동 63번지,서울특별시 서초구 잠원로 130,CJ빌리지,20050701,2021,10,4654.878415,30,250,공동주택,공동주택(아파트),17,29,127.008642,37.514897,POINT(127.008642332057 37.5148972246205),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\xf1b\x9...,973.351894
73,1165010800113010006,서울특별시 서초구 서초동 1301-6번지,서울특별시 서초구 서운로 221,래미안서초스위트,20090430,2022,4,4418.968058,13,279,공동주택,아파트,392,505,127.021226,37.502845,POINT(127.021226446301 37.5028447028925),b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x14++\x...,979.967854


In [25]:
from IPython.display import IFrame
# params
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']
center = [37.5150660594, 127.0180350423]

# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 14,
    tiles = tiles[2]
)


text=f"""지번:{df.iloc[i]['addressJibeon']} <br> 
      단지명 :{df.iloc[i]['danjiName']} <br> 
      사용승인일 :{df.iloc[i]['usePermissionDate']} <br> 
      전세전용평단가평균 : {df.iloc[i]['jeonseJeonyongUnitPriceAverage']} <br> 
      용적률 : {df.iloc[i]['floorAreaRatio']} <br>
      주용도 :{df.iloc[i]['mainPurposeCodeName']} <br>
      기타용도 : {df.iloc[i]['otherPurpose']} <br>
      세대수 : {df.iloc[i]['saedaeCount']} <br>
      총 주차대수 : {df.iloc[i]['totalParkingLotCount']} <br>
      거리 : {df.iloc[i]['distance']}""" 


text2 = f"""지번:{df.iloc[0]['addressJibeon']} <br> 
          단지명 :{df.iloc[0]['danjiName']} <br> 
          사용승인일 :{df.iloc[0]['usePermissionDate']} <br> 
          전세전용평단가평균 : {df.iloc[0]['jeonseJeonyongUnitPriceAverage']} <br> 
          용적률 : {df.iloc[0]['floorAreaRatio']} <br>
          주용도 :{df.iloc[0]['mainPurposeCodeName']} <br>
          기타용도 : {df.iloc[0]['otherPurpose']} <br>
          세대수 : {df.iloc[0]['saedaeCount']} <br>
          총 주차대수 : {df.iloc[0]['totalParkingLotCount']} <br>
          거리 : {df.iloc[0]['distance']}""" 


iframe = folium.IFrame(text)
iframe2 = folium.IFrame(text2)

# 입력받은 지번을 제외한 매물 마킹
for i in range(1,len(df)):
    folium.Marker(
        location = [df.iloc[i]['lat'], df.iloc[i]['lon']],      
        popup = folium.Popup(iframe, min_width =400, max_width = 400),
        tooltip = df.iloc[i]['danjiName'],
        icon = folium.Icon('blue', icon = icon[1])).add_to(m)
    
# 입력받은 지번 매물 마킹
folium.Marker(
    location = [lat,lon],
    popup = folium.Popup(iframe2, min_width = 500, max_widht = 500),
    tooltip = danjiName,
    icon = folium.Icon('red', icon = icon[0])).add_to(m)
 
    
for i in range(0, len(train)):
    folium.Marker(
        location = [train.iloc[i]['lat'], train.iloc[i]['lon']],
        popup = train.iloc[i]['lineName'],
        tooltip = train.iloc[i]['lineName'],
        icon = folium.Icon('green', icon = icon[1])).add_to(m)
m    


### pydeck 활용

## 추후 해야할것.
- 목적에 맞게 전달력있는 시각화방법
- 파라미터 수정을 통한 원하는 정보전달방법 
- 바로 아래 셀에서 좌표가 없는 아파트가 존재하는데 이유 찾기
- 좀더 랜더링을 빠르게할 수 없을지 생각해보기

In [6]:
# pydeck 연습용


# 입력된지번의 일정 반경내 아파트와 아파트 정보
sql = f"""
SELECT A.pnuCode, addressJibeon, addressRoad, danjiName, usePermissionDate, B.contractYear, B.contractMonth, B.jeonyongArea, B.jeonseJeonyongUnitPriceAverage, buildingAreaPerLandArea, floorAreaRatio, mainPurposeCodeName, otherPurpose, saedaeCount, totalParkingLotCount, lon, lat, coord, geom
FROM aide.apartment_danji_information AS A
JOIN (SELECT T1.*
    FROM(
         SELECT pnuCode,
                 contractYear,
                 contractMonth,
                 jeonyongArea,
                 supplyArea,
                 tradingJeonyongUnitPriceAverage,
                 jeonseJeonyongUnitPriceAverage,
                 wolseJeonyongUnitPriceAverage,
                 ROW_NUMBER() OVER(PARTITION BY pnuCode ,jeonyongArea ORDER BY contractYear DESC, contractMonth DESC) AS num
         FROM aide.apartment_danji_monthly_area_price
         WHERE pnuCode LIKE '{sidoCode}%' AND jeonseJeonyongUnitPriceAverage is not null
        ) AS T1
        WHERE T1.num = 1 
) AS B
on A.pnuCode = B.pnuCode 

"""
df= db.read_table(sql)


In [7]:
len(df[df['lon'].isna()])

414

In [8]:
df[df['lon'].isna()]

,pnuCode,addressJibeon,addressRoad,danjiName,usePermissionDate,contractYear,contractMonth,jeonyongArea,jeonseJeonyongUnitPriceAverage,buildingAreaPerLandArea,floorAreaRatio,mainPurposeCodeName,otherPurpose,saedaeCount,totalParkingLotCount,lon,lat,coord,geom
751,1114016200108510001,서울특별시 중구 신당동 851-1번지,서울특별시 중구 퇴계로90길 74,래미안하이베르(임대),20111028,2022,2,37.54,541.583665,23,227,공동주택,공동주택(아파트),161,161,NaN,NaN,None,None
759,1114016200108520001,서울특별시 중구 신당동 852-1번지,서울특별시 중구 청구로 64,청구e편한세상(임대),20111213,2022,3,35.79,519.374958,0,0,공동주택,공동주택(아파트),158,0,NaN,NaN,None,None
760,1114016200108520001,서울특별시 중구 신당동 852-1번지,서울특별시 중구 청구로 64,청구e편한세상(임대),20111213,2016,3,35.89,518.019937,0,0,공동주택,공동주택(아파트),158,0,NaN,NaN,None,None
1271,1117012200100070006,서울특별시 용산구 문배동 7-6번지,서울특별시 용산구 백범로90길 74,이안용산1차(102동),20051130,2022,2,79.63,2200.258842,44,423,공동주택,"공동주택(아파트),오피스텔",112,114,NaN,NaN,None,None
1272,1117012200100070006,서울특별시 용산구 문배동 7-6번지,서울특별시 용산구 백범로90길 74,이안용산1차(102동),20051130,2022,1,84.22,2071.512823,44,423,공동주택,"공동주택(아파트),오피스텔",112,114,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29677,1174010900104160012,서울특별시 강동구 천호동 416-12번지,서울특별시 강동구 구천면로24길 9,엘림스퀘어II,20190418,2021,8,14.92,3512.950613,60,540,공동주택,"도시형생활주택,근린생활시설,업무시설",143,81,NaN,NaN,None,None
29678,1174010900104160012,서울특별시 강동구 천호동 416-12번지,서울특별시 강동구 구천면로24길 9,엘림스퀘어II,20190418,2021,9,15.05,3690.178743,60,540,공동주택,"도시형생활주택,근린생활시설,업무시설",143,81,NaN,NaN,None,None
29679,1174010900104160012,서울특별시 강동구 천호동 416-12번지,서울특별시 강동구 구천면로24길 9,엘림스퀘어II,20190418,2022,4,17.07,3202.564065,60,540,공동주택,"도시형생활주택,근린생활시설,업무시설",143,81,NaN,NaN,None,None
29680,1174010900104160012,서울특별시 강동구 천호동 416-12번지,서울특별시 강동구 구천면로24길 9,엘림스퀘어II,20190418,2022,4,17.58,3109.844959,60,540,공동주택,"도시형생활주택,근린생활시설,업무시설",143,81,NaN,NaN,None,None


In [9]:
df = df.drop(index = df[df['lon'].isna()].index)

In [10]:
# import pydeck
import pydeck as pdk

#  wkt to shapely geometry
from shapely import wkt
df['coord'] = df['coord'].apply(wkt.loads)

In [11]:
# pydeck 을 사용하기 위해 리스트 형태로 변환
from tqdm import trange, notebook

# point to list
for i in notebook.tqdm(range(0, len(df))):
    x, y  = df['coord'].iloc[i].xy
    df['coord'].iloc[i] = [[x[0], y[0]]]
    
# # polygon to list
# for i in range(0, len(poly)):
#     poly['geometry'].iloc[i] = list(poly['geometry'].iloc[i].exterior.coords.xy)

  0%|          | 0/29499 [00:00<?, ?it/s]

C:\Users\JEONGS~1\AppData\Local\Temp/ipykernel_22856/3055160626.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['coord'].iloc[i] = [[x[0], y[0]]]


In [12]:
# 방해되는 컬럼 제거
df = df.drop(columns = 'geom')

In [13]:
# df = pd.read_json(df.to_json())

In [14]:
df['정규화평당가'] = df['jeonseJeonyongUnitPriceAverage'] / df['jeonseJeonyongUnitPriceAverage'].max()

In [ ]:
# scatterplotLayer
# Make layer
center = [df.iloc[0]['lat'], df.iloc[0]['lon']]
layer = pdk.Layer(
    'ScatterplotLayer', 
    df, 
    get_position='[lon, lat]',
    get_radius = 50,
    get_fill_color='[250*정규화평당가, 75*정규화평당가, 255]', # Set an RGBA value for fill
    pickable=True, # plot세부정보 확인
    auto_highlight=False
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=127.0180350423,
    latitude=37.5150660594,
    zoom=12)


# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
# heatmapLayer
layer = pdk.Layer(
    'HeatmapLayer',
    df,
    get_position='[lon, lat]',
    opacity=0.3# 불투명
)


view_state = pdk.ViewState(
    longitude=127.0180350423,
    latitude=37.5150660594,
    zoom=10)

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
layer = pdk.Layer(
    'CPUGridLayer', # 대용량 데이터의 경우 'GPUGridLayer'
    df,
    get_position='[lon, lat]',
    pickable=True,
    auto_highlight=True
)


view_state = pdk.ViewState(
    longitude=127.0180350423,
    latitude=37.5150660594,
    zoom=10)



r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
layer = pdk.Layer(
    'CPUGridLayer', # 대용량 데이터의 경우 'GPUGridLayer'
    df,
    get_position='[lon, lat]',
    pickable=True,
    auto_highlight=True,
    elevation_scale = 50,
    elevation_range=[0, 200],
     extruded=True
)


view_state = pdk.ViewState(
    longitude=127.0180350423,
    latitude=37.5150660594,
    zoom=10)


# layer.extruded = True;
# layer.get_elevation = 'saedaeCount';
# layer.elevation_scale = 50

view_state.bearing=15
view_state.pitch=45

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
# Define a layer to display on a map
layer = pdk.Layer(
    "HexagonLayer",
    df,
    get_position='[lon, lat]',
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0,200],
    extruded=True,
    coverage=1,
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=127.0180350423, latitude=37.5150660594, zoom=10, min_zoom=5, max_zoom=15, pitch=20, bearing=-27.36,
)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html("hexagon_layer.html")

In [ ]:
# pydeck POLYGON

# data load
sql3 = f"""
SELECT * FROM aide.polygon_bdong
WHERE bdongCode lIKE '11650%';
"""
poly = db.read_table(sql3)

#  wkt to shapely Polygon
from shapely import wkt
poly['geometry'] = poly['geometry'].apply(wkt.loads)


# polygon to list
for i in range(0, len(poly)):
    poly['geometry'].iloc[i] = list(poly['geometry'].iloc[i].exterior.coords.xy)[0]
    
# 임의의 컬럼생성
poly['정규화인구'] = [0.3213213,0.32132140, 0.12312516,0.6786547,0.645645,0.512534,0.3451346,0.754673456,0.413412,0.15153]

In [ ]:
# pydeck POLYGON

# data load
sql3 = f"""
SELECT * FROM aide.polygon_bdong
WHERE bdongCode lIKE '11650%';
"""
poly = db.read_table(sql3)

In [ ]:
#  wkt to shapely Polygon
from shapely import wkt
poly['geometry'] = poly['geometry'].apply(wkt.loads)
# 임의의 컬럼생성
poly['정규화인구'] = [0.3213213,0.32132140, 0.12312516,0.6786547,0.645645,0.512534,0.3451346,0.754673456,0.413412,0.15153]

In [ ]:
for i in range(0, len(poly)):
    poly['geometry'].iloc[i] = list(poly['geometry'].iloc[i].exterior.coords)
    

In [ ]:
poly

In [ ]:

# Make layer
layer = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    poly, # 시각화에 쓰일 데이터프레임
    get_polygon='geometry', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[255*정규화인구, 100*정규화인구, 255]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    pickable=True, # 지도와 interactive 한 동작 on
    auto_highlight=True # 마우스 오버(hover) 시 박스 출력
)

# Set the viewport location
center = [126.986, 37.565]
view_state = pdk.ViewState(
    longitude=center[0],
    latitude=center[1],
    zoom=10)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

## 전입전출 이동 시각화

In [31]:
# 모듈, 경로 설정
import pandas as pd
import pydeck as pdk
import geopandas as gpda
import config
import json
import requests
import sys
import folium

path = "C:\github/aide-analysis/"
sys.path.append(path)

path2 = "C:\github/prop-tech/"
sys.path.append(path2)
from src.sql import DatabaseHandler
db = DatabaseHandler()

from core.config import Config
config = Config()


In [32]:
from src.bigquery import bigquery_client, bigquery
from src.visualization import Pydeck
pydeck = Pydeck()
bq = bigquery_client
sql = f"""
SELECT *
FROM `aidepartners.aide.polygon_sigungu` AS sigungu
"""
sigungu = bq.query(sql).to_dataframe()

In [33]:
# from shapely import wkt
# sigungu['geometry'] = sigungu['geometry'].apply(wkt.loads)


In [34]:
# preprocessing
from shapely import wkt
s = sigungu['geometry']
s = s.apply(wkt.loads)

In [35]:
lst = []
for i in range(0,len(s)):
    a = s[i].centroid.x
    b = s[i].centroid.y
    lst.append([a,b])
lst

[[129.0943046029785, 35.12554958975966],
 [128.58532970374523, 35.83517827547346],
 [126.85673845568813, 35.09405737030621],
 [129.32819006482666, 35.5160516589541],
 [129.04458612987517, 35.12918660014703],
 [128.68564987549834, 35.93444401364048],
 [126.63954339158985, 37.483194105958006],
 [126.94946222857777, 35.11738405016881],
 [127.47506268010525, 36.32396533890902],
 [129.42606946719485, 35.52558000363764],
 [129.02346514041483, 35.22925310552127],
 [128.57720558270933, 35.92892764111476],
 [126.92546678530002, 35.193249128483494],
 [129.3798015100095, 35.610062400263125],
 [129.01491283727697, 35.102964521128314],
 [128.54969757064063, 35.87500168460091],
 [126.65636856981757, 37.557839070460936],
 [126.85072451951622, 35.1356966886166],
 [127.34512535641753, 36.28026323374417],
 [126.99596829344615, 37.56014287215071],
 [129.0315402231009, 35.10547795912984],
 [128.59360566085468, 35.8665352468194],
 [126.4828638496658, 37.46909379320836],
 [127.41105672133295, 36.28087229010

In [37]:
sigungu['x'] = ''
sigungu['y'] = ''

for i in range(0,len(sigungu)):
    sigungu['x'][i] = lst[i][0]
    sigungu['y'][i] = lst[i][1]
sigungu

,sigunguCode,sigunguName,geometry,x,y
0,26290,남구,MULTIPOLYGON (((129.1270230682668 35.090961166...,129.094305,35.12555
1,27200,남구,POLYGON ((128.5794505046589 35.857832407193854...,128.58533,35.835178
2,29155,남구,POLYGON ((126.90107370468607 35.15233821092302...,126.856738,35.094057
3,31140,남구,MULTIPOLYGON (((129.34931822705016 35.46703056...,129.32819,35.516052
4,26170,동구,POLYGON ((129.04264226677003 35.14589414794844...,129.044586,35.129187
...,...,...,...,...,...
245,41285,고양시일산동구,"POLYGON ((126.7776157073954 37.67899469095559,...",126.797569,37.679872
246,41287,고양시일산서구,POLYGON ((126.76703256685127 37.70716707003747...,126.727985,37.680204
247,36110,세종특별자치시,POLYGON ((127.17841156617698 36.59687459632547...,127.258722,36.560729
248,48125,창원시 마산합포구,MULTIPOLYGON (((128.5751820290527 35.062497749...,128.485599,35.135581


In [38]:
sigunguCode = sigungu[['sigunguCode','x','y']]

In [39]:
sql = f"""
SELECT insigunguCode, outsigunguCode, count(*) AS cnt
FROM aide.population_transfer
WHERE inYear = 2020 AND inSigunguCode LIKE '11%' AND outSigunguCode LIKE '11%' AND inSigunguCode != outSigunguCode
GROUP BY inSigunguCode, outSigunguCode
"""
df = db.read_table(sql)

In [40]:
inTransfer = pd.merge(df, sigunguCode, left_on = 'insigunguCode', right_on = 'sigunguCode', how = 'inner')
inTransfer = inTransfer[['insigunguCode','x','y','outsigunguCode','cnt']]
inTransfer = inTransfer.rename(columns = {'x':'in_x','y':'in_y'})
inTransfer = inTransfer[inTransfer['insigunguCode']=='11620']
inTransfer

,insigunguCode,in_x,in_y,outsigunguCode,cnt
480,11620,126.945336,37.467377,11110,323
481,11620,126.945336,37.467377,11140,303
482,11620,126.945336,37.467377,11170,633
483,11620,126.945336,37.467377,11200,571
484,11620,126.945336,37.467377,11215,826
485,11620,126.945336,37.467377,11230,651
486,11620,126.945336,37.467377,11260,486
487,11620,126.945336,37.467377,11290,731
488,11620,126.945336,37.467377,11305,473
489,11620,126.945336,37.467377,11320,380


In [41]:
inTransfer = pd.merge(inTransfer, sigunguCode, left_on = 'outsigunguCode', right_on = 'sigunguCode', how = 'inner')
inTransfer = inTransfer.rename(columns = {'x':'out_x','y':'out_y'})
inTransfer = inTransfer[['insigunguCode','in_x','in_y','outsigunguCode','out_x','out_y', 'cnt']]
inTransfer

,insigunguCode,in_x,in_y,outsigunguCode,out_x,out_y,cnt
0,11620,126.945336,37.467377,11110,126.977317,37.59492,323
1,11620,126.945336,37.467377,11140,126.995968,37.560143,303
2,11620,126.945336,37.467377,11170,126.979908,37.531386,633
3,11620,126.945336,37.467377,11200,127.041059,37.55103,571
4,11620,126.945336,37.467377,11215,127.085746,37.546722,826
5,11620,126.945336,37.467377,11230,127.054848,37.581957,651
6,11620,126.945336,37.467377,11260,127.092884,37.597819,486
7,11620,126.945336,37.467377,11290,127.017578,37.605701,731
8,11620,126.945336,37.467377,11305,127.011185,37.643477,473
9,11620,126.945336,37.467377,11320,127.032367,37.669105,380


In [42]:
pydeck.arclayer(df = inTransfer,width = 2 ,source_color_list =  '[110*cnt*0.001,60*cnt*0.001,50*cnt*0.001]',target_color_list = '[110*cnt*0.001,60*cnt*0.001,50*cnt*0.001]', center_xy_list = [126.986, 37.565])

In [19]:
pydeck.linelayer(df = inTransfer, width = 2, color = '[110*cnt*0.001,60*cnt*0.001,50*cnt*0.001]',center_xy_list = [126.986, 37.565])

In [79]:
sql =f"""
SELECT DISTINCT transfer.inHdongCode,
       st_x(st_centroid(ST_GeomFromText(hdong.geometry))) as x,
	   st_y(st_centroid(ST_GeomFromText(hdong.geometry))) as y
FROM aide.polygon_hdong AS hdong
RIGHT JOIN (SELECT inHdongCode
			FROM aide.population_transfer
			WHERE inYear = 2020 AND inHdongCode LIKE '11%' AND outHdongCode LIKE '11%') AS transfer
            ON hdong.hdongCode = transfer.inHdongCode;
"""
inTransaction = db.read_table(sql)

In [ ]:
sql2 = f"""
SELECT DISTINCT transfer2.outHdongCode,
       st_x(st_centroid(ST_GeomFromText(hdong.geometry))) as x,
	   st_y(st_centroid(ST_GeomFromText(hdong.geometry))) as y
FROM aide.polygon_hdong AS hdong
RIGHT JOIN (SELECT outHdongCode
			FROM aide.population_transfer
            WHERE inYear = 2020 AND outHdongCode LIKE '11%' AND outHdongCode LIKE '11%') AS transfer2
            ON hdong.hdongCode = transfer2.outHdongCode;
"""
outTransaction = db.read_table(sql2)

In [ ]:
df = pd.concat([inTransaction, outTransaction],axis =1)
df

In [65]:
df.columns = ['inHdongCode','in_x','in_y','outHdongCode','out_x','out_y']

In [66]:
data = df.copy()

In [67]:
from pyproj import Proj, transform
 
 
epsg5179 = Proj(init="epsg:2097")
wgs84 =Proj(init='epsg:4326')


C:\Users\jeongseokLim\anaconda3\envs\aide\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\jeongseokLim\anaconda3\envs\aide\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [68]:
data['in_x'], data['in_y'] = transform(epsg5179, wgs84, data['in_x'].tolist(), data['in_y'].tolist())
data['out_x'], data['out_y'] = transform(epsg5179, wgs84, data['out_x'].tolist(), data['out_y'].tolist())

C:\Users\JEONGS~1\AppData\Local\Temp/ipykernel_22856/673831523.py:1: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  data['in_x'], data['in_y'] = transform(epsg5179, wgs84, data['in_x'].tolist(), data['in_y'].tolist())
C:\Users\JEONGS~1\AppData\Local\Temp/ipykernel_22856/673831523.py:2: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  data['out_x'], data['out_y'] = transform(epsg5179, wgs84, data['out_x'].tolist(), data['out_y'].tolist())


In [69]:
data

,inHdongCode,in_x,in_y,outHdongCode,out_x,out_y
0,1111051500,126.968314,37.587407,1111053000,126.968041,37.576858
1,1111053000,126.968041,37.576858,1111055000,126.960453,37.599448
2,1111054000,126.979011,37.590763,1159066000,126.925922,37.509864
3,1111055000,126.960453,37.599448,1111051500,126.968314,37.587407
4,1111056000,126.967171,37.616714,1111070000,127.014759,37.579975
...,...,...,...,...,...,...
420,1174065000,127.125639,37.537115,1171053100,127.139579,37.497137
421,1174066000,127.131699,37.531154,1130562500,127.017401,37.648151
422,1174068500,127.143809,37.542476,1154569000,126.903822,37.443027
423,1174070000,127.145316,37.534266,1147059000,126.837714,37.524998


In [70]:
# import pydeck
import pydeck as pdk

In [71]:
df2 = data.copy()

In [73]:
layer = pdk.Layer(
    'ArcLayer',
    data = df2,
    get_source_position='[in_x, in_y]',
    get_target_position='[out_x, out_y]',
    get_width='1',
    get_source_color='[255, 255, 120]',
    get_target_color='[255, 0,255]',
    pickable=True,
    auto_highlight=True
)


center = [126.986, 37.565]
view_state = pdk.ViewState(
    longitude=center[0],
    latitude=center[1],
    zoom=10)
view_state.zoom = 12



m = pdk.Deck(layers=[layer], initial_view_state=view_state)
m.to_html()

### 이후 해야할 작업
- 시군구 법정동 혹은 행정동 단위로 POLYGON의 CENTROID를 좌표로 했기때문에 매우 복잡하다. 따라서 시도간의 CENTROID를 구하여 시도간의 이동을 알아보는것도 좋을거같다.

### 시군구경계 + 미분양